# ポアソン混合分布推定(変分推論)

このシートでは、潜在変数とパラメータを分けることによって、事後分布を近似することを目指す。

$$
p(S, \lambda, \pi | X) \approx q(S)q(\lambda, \pi)
$$

変分推論では、何らかのいい性質を持っている分布の中で、分布pと最も”近い”分布qを探すことを探すことが目的。
特に以下の仮定を置いたものを平均場近似という。
$$
p(z_1, z_2, z_3) \ approx q(z_1)q(z_2)q(z_3)
$$
この下で、各分解された近似分布$q(z_1), q(z_2), q(z_3)$をKLダイバージェンスが最も小さくなるように１つ１つ修正していくのが、平均場近似による変分推論の基本的なアルゴリズム。
例えば、q(z_2)とq(z_3)わかっているとすると、(期待値計算は、<>とかく)
)

$q_{opt.}(z_1) = \text{argmin KL}_{q(z_1)} [q(z_1)q(z_2)q(z_3) || p(z_1, z_2, z_3)]$
$$
\text{KL} [q(z_1)q(z_2)q(z_3) || p(z_1, z_2, z_3)] \\

= - <\ln \frac{p(z_1, z_2, z_3)}{q(z_1)q(z_2)q(z_3)}>_{1,2,3}

$$

In [4]:
import numpy as np
from scipy.special import psi
import matplotlib.pyplot as plt

In [21]:
# 混合ポアソン分布の生成
K = 3
N =300
lambda_true = np.array([1, 10, 20])
pi_true = np.array([0.2, 0.5, 0.3])

s_truth_nk = np.random.multinomial(n=1, pvals=pi_true, size=N)
_, s_truth_n = np.where(s_truth_nk == 1)
X = np.random.poisson(lam=lambda_true[s_truth_n], size=N)


In [22]:
# 事前分布の設定
a = 1.0
b = 1.0
alpha = np.repeat(2.0, K)

# 初期値
E_s_nk =np.random.rand(N, K)
E_s_nk =  E_s_nk / np.sum(E_s_nk, axis = 1, keepdims=True)

In [23]:
a_hat_k = np.sum(E_s_nk.T * X, axis=1) + a
b_hat_k = np.sum(E_s_nk, axis=0) + b
alpha_hat_k = np.sum(E_s_nk, axis=0) + alpha
E_lambda = a_hat_k/b_hat_k
E_pi = alpha_hat_k / np.sum(alpha_hat_k)


In [24]:
# 変分推論
Max_Iter = 300

for _ in range(Max_Iter):
    E_lambda = a_hat_k/b_hat_k
    E_ln_lambda = psi(a_hat_k) - np.log(b_hat_k)
    E_ln_pi = psi(alpha_hat_k) - psi(np.sum(alpha_hat_k))
    
    for n in range(N):
        eta= np.exp(X[n] * E_ln_lambda - E_lambda + E_ln_pi)
        eta = eta / np.sum(eta)
        
        E_s_nk[n] = eta.copy()
    
    a_hat_k = np.sum(E_s_nk.T * X, axis=1) + a
    b_hat_k = np.sum(E_s_nk, axis=0) + b
    
    alpha_hat_k = np.sum(E_s_nk, axis=0) + alpha

In [28]:
E_ln_lambda
a_hat_k
b_hat_k
alpha_hat_k

array([144.58999335,  63.77507572,  97.63493093])